## Pre-requisite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Zero/Zero"

/content/drive/MyDrive/Zero/Zero


In [ ]:
pip install adjustText kneed bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 44.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 15.1 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.1 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039297 sha256=11d06c30e29f0d96815118ef085733ebfa28caaf5bcb2c6714c5aeb6026a8db8
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-l

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Code

In [ ]:
import os
import json
import re
import textwrap
import pickle
import scipy
import ast
import math

import statsmodels as sm
import statsmodels.api as sm
import statsmodels as sm

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.cm import get_cmap
cmap_autumn = get_cmap('autumn')
cmap_winter = get_cmap('winter')

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from adjustText import adjust_text

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()

from kneed import KneeLocator

from bertopic import BERTopic

import gensim
from gensim import corpora, models
from gensim.models import word2vec, Word2Vec
import gensim.corpora as corpora

from transformers import pipeline
multilingual_classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

from datetime import datetime, timedelta
from itertools import islice
from collections import Counter, defaultdict

from wordcloud import WordCloud, STOPWORDS

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.neighbors import KernelDensity

from tqdm import tqdm

start = datetime.strptime("01-01-2018", "%d-%m-%Y")
end = datetime.strptime("31-12-2022", "%d-%m-%Y")
dates = [start + timedelta(days=x) for x in range(0, (end-start).days)]
x = np.linspace(1, len(dates), len(dates))

weight_for_gt = {
    "climate_engineering": 795000000,
    "geoengineering": 7530000,
    "solar_geoengineering": 1890000,
    "carbon_capture_and_removal": 55900000,
    "stratospheric_aerosol_injection": 289000,
    "cloud_brightening": 7000000,
    "ocean_fertilization": 8410000,
    "biochar": 24300000,
    "afforestation": 15000000,
    "reforestation": 30600000,
    "ecosystem_restoration": 139000000,
}

scaler = MinMaxScaler()

stopwords = set(STOPWORDS) | set(stopwords.words('english'))

stopwords_all = stopwords | set(["will", "may", "say", "says", "said", "told", "bbc", "video", "javascript", "sites",
                                 "mr", "year", "years", "people", "new", "number", "one", "two", "first", "last", "many", "much",
                                 "need", "want", "made", "make", "take", "time", "day", "part", "work", "play", "played",
                                 "u", "n't", "content", "enable", "browser",
                                 "us", "eu", "uk", "country", "ms.", "mr.", "even", "use", "go",
                                 "de", "la", "que", "en", "lo", "el", "includ", "include", "call",
                                 "http", "https", "link"])

stopwords_all_stem = set([p_stemmer.stem(token) for token in stopwords_all])

geoengineering_terms_category_dict = {}
geoengineering_terms_category = []
with open("geoengineering_terms_category.txt") as file:
    for line in file.readlines():
        thisone = " ".join([p_stemmer.stem(i) for i in line.strip().lower().split()])
        geoengineering_terms_category_dict[thisone] = line.strip().lower()
        geoengineering_terms_category.append(thisone)
geoengineering_terms_category_process = ["_".join(i.split()) for i in geoengineering_terms_category] + ["geoengin"]

geoengineering_terms_influencer_dict = {}
geoengineering_terms_influencer = []
with open("geoengineering_terms_influencer.txt") as file:
    for line in file.readlines():
        thisone = " ".join([p_stemmer.stem(i) for i in line.strip().lower().split()])
        geoengineering_terms_influencer_dict[thisone] = line.strip().lower()
        geoengineering_terms_influencer.append(thisone)
geoengineering_terms_influencer_process = ["_".join(i.split()) for i in geoengineering_terms_influencer]

geoengineering_terms_influencer_dict_whole = geoengineering_terms_influencer_dict.copy()
for key, value in geoengineering_terms_influencer_dict.items():
    geoengineering_terms_influencer_dict_whole["_".join(value.split())] = value.title()
    geoengineering_terms_influencer_dict_whole["_".join(key.split())] = value.title()
    geoengineering_terms_influencer_dict_whole[key] = value.title()

def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def plot_estimate_confidence(mean_conf, ax):
    categories = mean_conf.columns
    estimates = mean_conf.iloc[0]
    conf_intervals = list(mean_conf.iloc[1:, :].T.itertuples(index=False, name=None))

    for i, (estimate, interval) in enumerate(zip(estimates, conf_intervals)):
        ax.plot([interval[0], interval[1]], [i, i], color='skyblue')
        ax.plot(estimate, i, 'o', color='blue')
    ax.set_yticks(np.arange(len(categories)), categories)

def co_occurrence(sentences, window_size):
    d = defaultdict(int)
    vocab = set()

    for text in tqdm(sentences):
        for i in range(len(text)):
            token = text[i]
            vocab.add(token)
            next_token = text[i+1:i+1+window_size]
            for t in next_token:
                key = tuple(sorted([t, token]))
                d[key] += 1

    vocab = sorted(vocab)
    df = pd.DataFrame(data=np.zeros((len(vocab), len(vocab)), dtype=np.int16),
                      index=vocab,
                      columns=vocab)

    for key, value in tqdm(d.items()):
        df.at[key[0], key[1]] = value
        df.at[key[1], key[0]] = value
    return df

def tsne_plot(model, ax=None, random_state=1):
    labels, tokens = [], []

    for word in model.wv.index_to_key:
        tokens.append(model.wv[word])
        labels.append(word)

    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=random_state)
    new_values = tsne_model.fit_transform(np.array(tokens))

    x, y = [], []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])

    if not ax:
        fig, ax = plt.subplots(figsize=(12, 8))

    for i in tqdm(range(len(x))):
        if labels[i] in geoengineering_terms_category_process:
            ax.annotate(" ".join(labels[i].split("_")), xy=(x[i], y[i]), xytext=(5, 2),
                        textcoords='offset points', ha='right', va='bottom')
            ax.scatter(x[i], y[i], s=1)
        elif labels[i] in geoengineering_terms_influencer_process:
            ax.annotate(" ".join(labels[i].split("_")), xy=(x[i], y[i]), xytext=(5, 2),
                        textcoords='offset points', ha='right', va='bottom', color="red")
            ax.scatter(x[i], y[i], s=1)
        # else:
        #     ax.scatter(x[i], y[i], s=0.05)

def pickup_data(root):
    if root == "bbc":
        title_split = " - BBC "
    elif root == "nytimes":
        title_split = " - The New York Times"
    titles = []
    contents = []
    texts = []
    texts_multi = []
    list_root = [i for i in os.listdir(root) if i [0] != "."]
    years = sorted(list_root, key=int)
    for year in years:
        months = sorted(os.listdir(os.path.join(root, year)), key=int)
        for month in months:
            days = sorted(os.listdir(os.path.join(root, year, month)), key=int)
            for day in days:
                date = os.path.join(year, month, day)
                with open(os.path.join(root, year, month, day, 'articles')) as txt:
                    articles = json.loads(txt.read())
                    for article in articles["articles"]:
                        title = article["title"].split(title_split)[0]
                        content = article["content"]
                        description = article["description"]
                        if description is None:
                            description_input = " "
                        else:
                            description_input = " " + description + " "

                        titles += [[date, title]]
                        contents += [[date, content]]
                        texts += [[date, title + description_input + content]]
                        texts_multi += [[date, title, description, content]]
    return titles, contents, texts, texts_multi

def lower_df_data(df, index=[1,2,3]):
    if type(index) == int:
        index = [index]
    for i in index:
        df[i] = df[i].str.lower()
    return df

def choose_df_data(df, index=[1,2,3], words_need=""):
    if type(index) == int:
        index = [index]
    df_contains_temp = False
    for i in index:
        df_contains_temp = df_contains_temp | df[i].str.contains(words_need)
    df = df[df_contains_temp]
    return df

def stack_df_data(df, index=[1,2,3]):
    if type(index) == int:
        index = [index]
    df_stack = []
    for _, row in df.iterrows():
        for i in index:
            df_stack.append([row[0], row[i]])
    df_stack = pd.DataFrame(df_stack)
    return df_stack

def word_process(text):
    if text is None:
        return [""]
    text_token = nltk.word_tokenize(text)
    text_token_nonstop = [token for token in text_token if (token[0].isalpha()) and (token not in stopwords_all)]
    text_token_nonstop_stem = [p_stemmer.stem(token) for token in text_token_nonstop]
    text_token_nonstop_stem_combine = " ".join(text_token_nonstop_stem)
    for category in geoengineering_terms_category:
        if category in text_token_nonstop_stem_combine:
            text_token_nonstop_stem_combine = text_token_nonstop_stem_combine.replace(category, "_".join(category.split()))
    for category in geoengineering_terms_influencer:
        if category in text_token_nonstop_stem_combine:
            text_token_nonstop_stem_combine = text_token_nonstop_stem_combine.replace(category, "_".join(category.split()))
    return text_token_nonstop_stem_combine.split()

def process_df_data(df, index=[1,2,3]):
    if type(index) == int:
        index = [index]
    for i in index:
        df[i] = df[i].apply(lambda x: word_process(x))
    return df

def pre_workflow(news, words_need, process=True):
    _, _, _, data = pickup_data(news)
    data = pd.DataFrame(data)
    data[0]= pd.to_datetime(data[0]).dt.strftime('%Y/%m/%d')
    data = lower_df_data(data)
    data = choose_df_data(data, words_need=words_need)
    data = stack_df_data(data)
    if process:
        data = process_df_data(data, index=1)
    return data

def time_range_split(start, end, num):
    if num == 5:
        return ["2018/01/01", "2018/12/31",
                "2019/01/01", "2019/12/31",
                "2020/01/01", "2020/12/31",
                "2021/01/01", "2021/12/31",
                "2022/01/01", "2022/12/31"]
    fmt = "%Y/%m/%d"
    start = datetime.strptime(start, fmt)
    end = datetime.strptime(end, fmt)
    diff = (end - start) / num
    split_points = []
    for i in range(num):
        split_points.append((start + diff * i).strftime(fmt))
        if i > 0:
            split_points.append((start + diff * i + timedelta(days = 1)).strftime(fmt))
    split_points.append(end.strftime(fmt))
    return split_points

def split_df_data(data, num, time_col_number = 0):
    split_points = time_range_split(data.iloc[0, time_col_number], data.iloc[-1, time_col_number], num)
    data_split = []
    for i in range(len(split_points) // 2):
        data_split.append(data[(data.iloc[:, time_col_number] >= split_points[i*2]) & (data.iloc[:, time_col_number] <= split_points[i*2+1])])
    return data_split, split_points

def draw_wordcloud(words, stopwords = stopwords, ax=None, width = 800, height = 400):
    wordcloud = WordCloud(width = width, height = height,
                    background_color ='white',
                    stopwords = stopwords,
                    collocation_threshold = 1000,
                    min_font_size = 5).generate(words)

    if ax is None:
        _, ax = plt.subplots(figsize = (8, 8), facecolor = None)
        ax.imshow(wordcloud)
    else:
        ax.imshow(wordcloud)

    ax.axis("off");

def draw_wordcloud_df_data(data, ax=None, width = 800, height = 400, text_col=1):
    words = " ".join(data[text_col].apply(lambda x: " ".join(x)))
    draw_wordcloud(words, stopwords = stopwords_all_stem, ax=ax, width = width, height = height)

def draw_wordcloud_list_data(data_splits, data_split_points, title="", text_col=1):
    fig, axes = plt.subplots(nrows=1, ncols=len(data_splits), figsize=(30,10))
    for i, data_split in enumerate(data_splits):
        draw_wordcloud_df_data(data_split, ax = axes[i], width = 800, height = 1600, text_col=text_col)
        axes[i].set_title("%s ~ %s" % (data_split_points[i*2], data_split_points[i*2+1]))
    fig.suptitle(title)
    fig.tight_layout();

def draw_termfreq_df_data(data, num_terms, ax=None, text_col=1):
    words = list([a for b in data[text_col].tolist() for a in b])
    words_termfreq = Counter(words).most_common(num_terms)
    words_termfreq = pd.DataFrame(words_termfreq).set_index(0)[::-1]
    words_termfreq.plot.barh(ax=ax)

def draw_termfreq_list_data(data_splits, data_split_points, num_terms, title="", text_col=1):
    fig, axes = plt.subplots(nrows=1, ncols=len(data_splits), figsize=(20,8))
    for i, data_split in enumerate(data_splits):
        draw_termfreq_df_data(data_split, num_terms, ax=axes[i], text_col=text_col)
        axes[i].legend().set_visible(False)
        axes[i].set_title("%s ~ %s" % (data_split_points[i*2], data_split_points[i*2+1]))
        axes[i].set_ylabel("")
    fig.suptitle(title)
    fig.tight_layout();

def draw_lda_df_data(data, num_topics, num_words, show_formatted=False):
    dictionary = corpora.Dictionary(data[1])
    corpus = [dictionary.doc2bow(i) for i in data[1]]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, random_state=706)
    if show_formatted:
        print(ldamodel.print_topics(num_topics=num_topics, num_words=num_words))
    lda_results = ldamodel.show_topics(num_topics=num_topics, num_words=num_words, formatted=False)
    return lda_results

def draw_lda_list_data(data_splits, data_split_points, num_topics, num_words, title=""):
    print(title, end="\n\n")
    for i, data_split in enumerate(data_splits):
        print("%s ~ %s" % (data_split_points[i*2], data_split_points[i*2+1]))
        lda_results = draw_lda_df_data(data_split, num_topics, num_words)
        for topic in lda_results:
            print()
            for token in topic[1]:
                print(token[0], token[1])
        print("\n\n\n")

In [ ]:
with open('geoengineering_terms_influencer.txt') as file:
    geoengineering_terms = [i.strip().lower() for i in file.readlines()]
words_need = "|".join(geoengineering_terms)
words_need

'biodiversity|deforestation|natural disaster|sea level rise|plastic pollution|ocean acidification|air pollution|global warming|overfishing|climate change'

In [ ]:
# bbc_unprocess = pre_workflow("bbc", words_need, process=False)
# bbc_unprocess.to_csv("PPT/bbc_unprocess.csv", index=False)

# nytimes_unprocess = pre_workflow("nytimes", words_need, process=False)
# nytimes_unprocess.to_csv("PPT/nytimes_unprocess.csv", index=False)

In [ ]:
bbc_unprocess = pd.read_csv('PPT/bbc_unprocess.csv', names=[0,1], header=0)
bbc_unprocess[1] = bbc_unprocess[1].fillna("")
nytimes_unprocess = pd.read_csv('PPT/nytimes_unprocess.csv', names=[0,1], header=0)
nytimes_unprocess[1] = nytimes_unprocess[1].fillna("")

In [ ]:
bbc_unprocess_agg = bbc_unprocess.groupby(bbc_unprocess.index // 3)[1].agg(lambda x: x.iloc[0]+ " \n " + x.iloc[1] + " \n " + x.iloc[2]).to_frame()
bbc_unprocess_agg_time = bbc_unprocess.groupby(bbc_unprocess.index // 3)[0].agg(lambda x: x.iloc[0]).to_frame()
bbc_unprocess_agg_df = pd.concat([bbc_unprocess_agg_time, bbc_unprocess_agg], axis=1)
bbc_unprocess_agg_df[2] = "BBC"

nytimes_unprocess_agg = nytimes_unprocess.groupby(nytimes_unprocess.index // 3)[1].agg(lambda x: x.iloc[0]+ " \n " + x.iloc[1] + " \n " + x.iloc[2]).to_frame()
nytimes_unprocess_agg_time = nytimes_unprocess.groupby(nytimes_unprocess.index // 3)[0].agg(lambda x: x.iloc[0]).to_frame()
nytimes_unprocess_agg_df = pd.concat([nytimes_unprocess_agg_time, nytimes_unprocess_agg], axis=1)
nytimes_unprocess_agg_df[2] = "NYTimes"

data_unprocess_agg_df = pd.concat([bbc_unprocess_agg_df, nytimes_unprocess_agg_df]).sort_values(0)

In [ ]:
bbc_unprocess[0] = pd.to_datetime(bbc_unprocess[0])
nytimes_unprocess[0] = pd.to_datetime(nytimes_unprocess[0])

In [ ]:
bbc_unprocess.shape[0]/3

2996.0

In [ ]:
(nytimes_unprocess.groupby([nytimes_unprocess[0].dt.year]).agg('count') /3).median()


0    2619.0
1    2619.0
dtype: float64

In [ ]:
data_unprocess_agg_df

,0,1,2
0,2018/01/01,"fighting climate change, one laundry load at a...",NYTimes
2,2018/01/01,the hunt for ‘fire cats’ amid northern califor...,NYTimes
3,2018/01/01,"with disney deal looming, murdoch’s empire is ...",NYTimes
4,2018/01/01,2 american families are mourned after plane cr...,NYTimes
5,2018/01/01,"the trump effect: business, anticipating less ...",NYTimes
...,...,...,...
2994,2022/12/31,new year honours 2023: brian may and lionesses...,BBC
13032,2022/12/31,opinion | biden set an ambitious goal for natu...,NYTimes
2993,2022/12/31,happisburgh: the norfolk village crumbling int...,BBC
2995,2022/12/31,the uk's problems will not all go away in 2023...,BBC


### Topic modeling

In [ ]:
data_topic_model = BERTopic(verbose=False)
data_topics, data_probs = data_topic_model.fit_transform(data_unprocess_agg_df[1])
data_all_topics = data_topic_model.get_topic_info()
data_document_topics = data_topic_model.get_document_info(data_unprocess_agg_df[1])
data_unprocess_agg_df[3] = data_document_topics["Name"].tolist()
data_unprocess_agg_df[4] = data_document_topics["Probability"].tolist()
data_unprocess_agg_df.to_csv('PPT/data_unprocess_agg_df_berttopics.csv', index=False)
data_all_topics.to_csv("PPT/data_all_topics.csv")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
data_unprocess_agg_df

,0,1,2,3,4
0,2018/01/01,"fighting climate change, one laundry load at a...",NYTimes,242_fungi_earthworms_soil_kiers,0.913642
2,2018/01/01,the hunt for ‘fire cats’ amid northern califor...,NYTimes,7_fire_fires_california_wildfires,0.942400
3,2018/01/01,"with disney deal looming, murdoch’s empire is ...",NYTimes,-1_the_and_of_to,0.000000
4,2018/01/01,2 american families are mourned after plane cr...,NYTimes,-1_the_and_of_to,0.000000
5,2018/01/01,"the trump effect: business, anticipating less ...",NYTimes,-1_the_and_of_to,0.000000
...,...,...,...,...,...
2994,2022/12/31,new year honours 2023: brian may and lionesses...,BBC,-1_the_and_of_to,0.000000
13032,2022/12/31,opinion | biden set an ambitious goal for natu...,NYTimes,15_species_biodiversity_wildlife_endangered,0.757703
2993,2022/12/31,happisburgh: the norfolk village crumbling int...,BBC,163_sea_coastal_flooding_defences,1.000000
2995,2022/12/31,the uk's problems will not all go away in 2023...,BBC,106_sunak_rishi_sunaks_downing,0.176472


In [ ]:
data_unprocess_agg_df_berttopics = pd.read_csv('PPT/data_unprocess_agg_df_berttopics.csv', names=[0, 1, 2, 3, 4], header=0)

In [ ]:
data_unprocess_agg_df_berttopics_lda = data_unprocess_agg_df_berttopics[[0, 3, 4]]
data_unprocess_agg_df_berttopics_lda_pivot = pd.pivot_table(data_unprocess_agg_df_berttopics_lda, index=0, columns=3, values=4, aggfunc='sum').fillna(0)
data_unprocess_agg_df_berttopics_lda_pivot.index = pd.to_datetime(data_unprocess_agg_df_berttopics_lda_pivot.index)
data_unprocess_agg_df_berttopics_lda_pivot_normal = data_unprocess_agg_df_berttopics_lda_pivot.T.apply(normalize_data).apply(lambda x: x / x.sum()).T
data_unprocess_agg_df_berttopics_lda_pivot_normal = data_unprocess_agg_df_berttopics_lda_pivot_normal.reindex(sorted(data_unprocess_agg_df_berttopics_lda_pivot_normal.columns, key=lambda x: int(x.split("_")[0])), axis=1)
data_unprocess_agg_df_berttopics_lda_pivot_normal.to_csv("PPT/data_unprocess_agg_df_berttopics_lda_pivot_normal.csv")